In [1]:
#%load_ext tensorboard
#%tensorboard --logdir logs/fit

## 1. Preprocesamiento de los datos

### 1.1 Preparación del entorno 

Aquí lo que hago es preaprar los imports principales y las variables de entorno. Ahora están comentadas porque como te dije en el correo estaba teniendo problemas con la versión de Cuda y Keras. Ahora lo he solucionado, pero basicamente lo que pasaba es que estaba usando una versión de Keras (Keras 3) la cual no es compatible con transfomers, al final lo soluciones activando  la variable de entorno TF_USE_LEGACY_KERAS e instalando en mi env de conda tf_keras para tener Keras 2. 

La variable TF_ENABLE_XLA la desactivé porque se supone que podía ser una causa de un problema de OOM que estaba teniendo durante el entrenamiento, pero al final resultó ser un problema con las versiones de las librerías.

La variables de entorno que tienen que ver con CUDA era porque creía que mi entorno virtual de conda (el cual al final lo he tenido que meter en un WLS2 con ubuntu porque en windows estaba teniendo problemas de compatibilidad peores, estaba cogienod) estaba cogiendo la versión de CUDA que no era, porque tenía varias instaladas. Pero era más un fallo de configuración del entorno que eso.

También he añadido una sección en la que controlo si el dispositivo con el que se va a entrenar es la GPU, pongo un creciomiento progresivo en el uso de memoria para evitar sobrecarga y también un límite para evitar de nuevo el OOM.

La línea tf.config.optimizer.set_jit(False) la usaba cuando tenía desactivado el XLA para evitar así que compilase por XLA y evitar posibles problemas de rendimiento, pero ese no era el problema. 

In [2]:
import os
os.environ['TF_USE_LEGACY_KERAS'] = '1'

import tensorflow as tf
info = tf.sysconfig.get_build_info()
print("CUDA:",   info["cuda_version"])
print("cuDNN:",  info["cudnn_version"])

from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy('mixed_float16')
print("Mixed precision policy:", mixed_precision.global_policy())

import random
import numpy as np
import transformers

# Fijamos la semilla para reproducibilidad
SEED = 42
#random.seed(SEED)
#np.random.seed(SEED)
#tf.random.set_seed(SEED)
tf.keras.utils.set_random_seed(SEED)
os.environ['PYTHONHASHSEED'] = str(SEED) 
tf.config.experimental.enable_op_determinism() # Para evitar problemas de determinismo en TensorFlow 

# Hiperparámetros
NUM_LABELS = 44  # 43 emociones + 1 sin emoción

MODEL_NAME = "beomi/KcELECTRA-base" #"beomi/KcELECTRA-base" "monologg/kobert"
MAX_LENGTH = 512 # Longitud máxima de las secuencias
BATCH_SIZE = 32

DROPOUT_RATE   = 0.3      
L2_REG         = 1e-5         
UNFREEZE_EPOCH = 3        

EPOCHS = 10

BASE_LR = 1e-4
FT_LR = 2e-5
WEIGHT_DECAY   = 0.01    # Decaimiento de pesos 
BETA_1         = 0.9     # Parámetro β₁ de AdamW
BETA_2         = 0.999   # Parámetro β₂ de AdamW
EPSILON        = 1e-6    # Epsilon de AdamW para estabilidad numérica

# Forzar el uso de la GPU y activamos el crecimiento de memoria y la limitamo para evitar el OOM
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)
    tf.config.experimental.set_virtual_device_configuration(
        gpus[0],
        [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=14336)]
    )
    device_name = "GPU"
else:
    device_name = "CPU"

print("Dispositivo:", device_name)
    
print("TensorFlow version:", tf.__version__)
print("Transformers version:", transformers.__version__)

2025-06-16 18:35:24.232024: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-16 18:35:24.417850: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750091724.491713   64353 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750091724.514403   64353 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1750091724.677110   64353 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

CUDA: 12.5.1
cuDNN: 9
INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA GeForce RTX 4090 Laptop GPU, compute capability 8.9
Mixed precision policy: <Policy "mixed_float16">


/home/carlo/miniconda3/envs/tf-gpu-v2/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Dispositivo: GPU
TensorFlow version: 2.19.0
Transformers version: 4.52.1


### 1.2 Previsualización de los datos de entrenamiento, validación y test

Aquí he cargado los datasets de manera manual, pensé en hacerlo descargando directamente desde huggingface como hacen en el código de KOTE, pero ya que tenía los archivos quise probar a hacerlo así.

In [3]:
import pandas as pd
#from datasets import load_dataset

# Cargo los datasets en local pero también podría ser desde HuggingFace como en el notebook que da KOTE: dataset = load_dataset("searle-j/kote")

train_path = "train.tsv"
val_path   = "val.tsv"    
test_path  = "test.tsv"

columns = ["id", "text", "labels"] 
df_train = pd.read_csv(train_path, sep="\t", header=None, names=columns)
df_val   = pd.read_csv(val_path,   sep="\t", header=None, names=columns)
df_test  = pd.read_csv(test_path,  sep="\t", header=None, names=columns)

print(f"Ejemplos cargados de Train: {len(df_train)}, Val: {len(df_val)}, Test: {len(df_test)}")

df_train.head(3)

Ejemplos cargados de Train: 40000, Val: 5000, Test: 5000


,id,text,labels
0,39087,내가 톰행크스를 좋아하긴 했나보다... 초기 영화 빼고는 다 봤네.,"2,13,15,16,29,39"
1,30893,"정말 상상을 초월하는 무개념 진상들 상대하다 우울증, 공항장애 걸리는 공무원 많아요...","0,5,7,10,19,22,29,35,36,38"
2,45278,"새로운 세상과 조우한 자의 어린아이 같은 반응, 어쩌면 회복된 것은 눈이 아닌 순수...","1,2,7"


#### 1.2.1 Control de sesgos de género

In [4]:
import re

# Definimos el mapeo de términos de género
gender_map = {
    "여자":       "남자",      # mujer -> hombre
    "남자":       "여자",      # hombre -> mujer
    "여성":       "남성",      # femenino -> masculino
    "남성":       "여성",      # masculino -> femenino

    "아버지":     "어머니",    # padre -> madre
    "어머니":     "아버지",    # madre -> padre
    "아들":       "딸",        # hijo -> hija
    "딸":         "아들",      # hija -> hijo
    "남편":       "아내",      # esposo -> esposa
    "아내":       "남편",      # esposa -> esposo
    "오빠":       "언니",      # hermano mayor (hablante femenino) -> hermana mayor
    "언니":       "오빠",      # hermana mayor -> hermano mayor (hablante femenino)
    "형":         "누나",      # hermano mayor (hablante masculino) -> hermana mayor
    "누나":       "형",        # hermana mayor -> hermano mayor (hablante masculino)

    "남자친구":   "여자친구",  # novio -> novia
    "여자친구":   "남자친구",  # novia -> novio
    "총각":       "처녀",      # soltero -> soltera
    "처녀":       "총각",      # soltera -> soltero

    "왕자":       "공주",      # príncipe -> princesa
    "공주":       "왕자",      # princesa -> príncipe
    "왕":         "여왕",      # rey -> reina
    "여왕":       "왕",        # reina -> rey

    "남배우":     "여배우",    # actor -> actriz
    "여배우":     "남배우",    # actriz -> actor

    "그는":       "그녀는",    # él (sujeto) -> ella (sujeto)
    "그녀는":     "그는",      # ella (sujeto) -> él (sujeto)
    "그를":       "그녀를",    # lo/le (objeto) -> la/le (objeto)
    "그녀를":     "그를",      # la/le (objeto) -> lo/le (objeto)
    "그의":       "그녀의",    # su (masculino) -> su (femenino)
    "그녀의":     "그의",      # su (femenino) -> su (masculino)

    "남성적":     "여성적",    # masculino (adjetivo) -> femenino (adjetivo)
    "여성적":     "남성적",    # femenino (adjetivo) -> masculino (adjetivo)
}


# Identificamos las filas cuyos textos contienen alguna clave de gender_map
pattern = "|".join(map(re.escape, gender_map.keys()))
mask = df_train['text'].str.contains(pattern)

# Creamos un DataFrame con las filas a gender-swappear
df_swapped = df_train[mask].copy()

# Aplicamos el reemplazo en la columna de texto
def swap_gender_tokens(txt):
    for src, tgt in gender_map.items():
        txt = txt.replace(src, tgt)
    return txt

df_swapped['text'] = df_swapped['text'].apply(swap_gender_tokens)

# Concatenamos y barajamos el DataFrame resultante antes del split
df_train = pd.concat([df_train, df_swapped], ignore_index=True)
df_train = df_train.sample(frac=1, random_state=42).reset_index(drop=True)

print(f"Añadidos {len(df_swapped)} ejemplos de género intercambiado. Nuevo tamaño de df_train: {len(df_train)}")

Añadidos 3422 ejemplos de género intercambiado. Nuevo tamaño de df_train: 43422


### 1.3 Binarización de las etiquetas

In [5]:
from sklearn.preprocessing import MultiLabelBinarizer

# Para convertir la columna labels de string a lista de ints
def parse_labels(label_str):
    if pd.isna(label_str) or label_str == "":
        return []
    return [int(x) for x in label_str.split(",")]

train_label_lists = df_train["labels"].apply(parse_labels)
val_label_lists   = df_val["labels"].apply(parse_labels)
test_label_lists  = df_test["labels"].apply(parse_labels)

# Pasamos la lisa de etiquetas a un formato multi-hot
# (una lista de listas de etiquetas, donde cada lista tiene el mismo tamaño que el número total de etiquetas)
mlb = MultiLabelBinarizer(classes=list(range(NUM_LABELS)))
mlb.fit(train_label_lists)

y_train = mlb.transform(train_label_lists)
y_val   = mlb.transform(val_label_lists)
y_test  = mlb.transform(test_label_lists)

print("Tamaño de y_train:", y_train.shape)
print("Ejemplo de vector de etiquetas (multi-hot) para una muestra:\n", y_train[0])

Tamaño de y_train: (43422, 44)
Ejemplo de vector de etiquetas (multi-hot) para una muestra:
 [0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 1 0 0 0 1 0 1 0 0 0 1 1 0 0
 0 0 0 0 0 1 1]


### 1.4 Revisión de los comentarios y pasarlos a listas

In [6]:
# Pasamos los comentarios también a listas
train_texts = df_train["text"].tolist()
val_texts   = df_val["text"].tolist()
test_texts  = df_test["text"].tolist()

print("Texto de ejemplo:", train_texts[0])
print("Etiquetas de ejemplo:", train_label_lists.iloc[0])
print("Vector multi-hot:", y_train[0])

Texto de ejemplo: 밤을 꼬박 샜다..  오늘이 일요일이라 다행이다.
Etiquetas de ejemplo: [4, 14, 22, 23, 27, 29, 33, 34, 42, 43]
Vector multi-hot: [0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 1 0 0 0 1 0 1 0 0 0 1 1 0 0
 0 0 0 0 0 1 1]


### 1.5 Definición del tokenizador

In [7]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)

## 2. Definición del modelo 

### 2.1 Carga del modelo preentrenado de transformer

In [8]:
from transformers import TFAutoModel, AutoConfig

config = AutoConfig.from_pretrained(MODEL_NAME)
transformer_model = TFAutoModel.from_pretrained(MODEL_NAME, config=config)

I0000 00:00:1750091730.767420   64353 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 14336 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4090 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.9
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFElectraModel: ['discriminator_predictions.dense.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.bias', 'electra.embeddings.position_ids', 'discriminator_predictions.dense_prediction.weight']
- This IS expected if you are initializing TFElectraModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFElectraModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceCla

### 2.2 Pooling de Representaciones y Clasificación

In [9]:
from tensorflow.keras import Input, Model, regularizers
from tensorflow.keras.layers import Dropout, Dense

def build_model():
    # 1) Carga limpia del transformer
    config = AutoConfig.from_pretrained(MODEL_NAME)
    transformer = TFAutoModel.from_pretrained(MODEL_NAME, config=config)
    # 2) Entradas
    input_ids     = tf.keras.Input(shape=(None,), dtype=tf.int32, name="input_ids")
    attention_mask = tf.keras.Input(shape=(None,), dtype=tf.int32, name="attention_mask")
    # 3) Forward + pooling (aquí usamos mean pooling; puedes cambiar a CLS token si lo prefieres)
    outputs = transformer(input_ids=input_ids, attention_mask=attention_mask)
    sequence_output = outputs.last_hidden_state
    pooled_output = tf.reduce_mean(sequence_output, axis=1)
    # 4) Cabeza de clasificación
    x = tf.keras.layers.Dense(256, activation="relu",
                              kernel_regularizer=tf.keras.regularizers.l2(L2_REG)
                             )(pooled_output)
    x = tf.keras.layers.Dropout(DROPOUT_RATE)(x)
    logits = tf.keras.layers.Dense(NUM_LABELS, activation="sigmoid",
                                   kernel_regularizer=tf.keras.regularizers.l2(L2_REG)
                                  )(x)
    model = tf.keras.Model(inputs=[input_ids, attention_mask], outputs=logits)

    return model, transformer

In [10]:
# Limpiamos la sesión de Keras para evitar el OOM
tf.keras.backend.clear_session()

## 3. Entrenamiento

### 3.1 Guardado del modelo y callbacks

In [11]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, TensorBoard
import re
from datetime import datetime

# Directorio para guardar el mejor modelo
#checkpoint_path = "best_model.h5"
clean_name = re.sub(r'[^A-Za-z0-9._-]', '_', MODEL_NAME)
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
checkpoint_path = f"{clean_name}_{timestamp}.h5"

callbacks = [
    EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True, verbose=1), # Así evitamos el overfitting
    ModelCheckpoint(filepath=checkpoint_path, monitor='val_loss', save_best_only=True, verbose=1), # Nos quedamos con el mejor modelo
    ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=1, verbose=1), # Reducimos el learning rate si no mejora la val_loss
]

### 3.2 Entrenamiento

In [12]:
# ── Función generadora de datos para tf.data.Dataset ──
def data_generator(texts, labels):
    for text, label in zip(texts, labels):
        # Tokeniza sin padding fijo; el padding lo hará el batch
        enc = tokenizer(
            text,
            truncation=True,
            max_length=MAX_LENGTH,
            padding=False
        )
        input_ids     = enc["input_ids"]
        attention_mask = enc["attention_mask"]
        yield (input_ids, attention_mask), label

# ── Tipos y formas de la salida para from_generator ──
output_types = (
    (tf.int32, tf.int32),  # (input_ids, attention_mask)
    tf.int32              # label multietiqueta
)

output_shapes = (
    (tf.TensorShape([None]), tf.TensorShape([None])),  # secuencia variable
    tf.TensorShape([NUM_LABELS])                       # vector de etiquetas
)

In [ ]:
# ── 1) Combinar train + val ──
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
from tensorflow.keras.metrics import AUC
from tensorflow.keras.optimizers.experimental import AdamW
from tensorflow.keras.mixed_precision import LossScaleOptimizer


texts_all  = train_texts + val_texts
labels_all = np.vstack([y_train, y_val])  # shape = (len(train)+len(val), NUM_LABELS)

# ── 2) Preparar CV ──
mskf = MultilabelStratifiedKFold(n_splits=5, shuffle=True, random_state=42)
fold_aucs = []

# ── 3) Loop de folds ──
for fold, (train_idx, val_idx) in enumerate(mskf.split(texts_all, labels_all), start=1):
    print(f"\n>>> Fold {fold}")

    # Partición de textos y etiquetas
    X_tr = [texts_all[i] for i in train_idx]
    y_tr = labels_all[train_idx]
    X_va = [texts_all[i] for i in val_idx]
    y_va = labels_all[val_idx]

    # Construir tf.data.Dataset para este fold
    train_ds = (
        tf.data.Dataset
          .from_generator(lambda: data_generator(X_tr, y_tr),
                          output_types=output_types,
                          output_shapes=output_shapes)
          .shuffle(len(X_tr), seed=42)
          .padded_batch(
              BATCH_SIZE,
              padded_shapes=(([None], [None]), [NUM_LABELS]),
              padding_values=((tokenizer.pad_token_id, 0), 0)
          )
          .prefetch(tf.data.AUTOTUNE)
    )
    val_ds = (
        tf.data.Dataset
          .from_generator(lambda: data_generator(X_va, y_va),
                          output_types=output_types,
                          output_shapes=output_shapes)
          .padded_batch(
              BATCH_SIZE,
              padded_shapes=(([None], [None]), [NUM_LABELS]),
              padding_values=((tokenizer.pad_token_id, 0), 0)
          )
          .prefetch(tf.data.AUTOTUNE)
    )

    # Construir modelo desde cero
    model, transformer_model = build_model()

    # — Pre‐entrenamiento de la cabeza —
    opt1 = AdamW(
        learning_rate=BASE_LR,
        weight_decay=WEIGHT_DECAY,
        beta_1=BETA_1,
        beta_2=BETA_2,
        epsilon=EPSILON
    )
    opt1 = LossScaleOptimizer(opt1)

    transformer_model.trainable = False
    model.compile(
        optimizer=opt1,
        loss="binary_crossentropy",
        metrics=[AUC(name="AUC", multi_label=True)]
    )
    model.fit(
        train_ds,
        validation_data=val_ds,
        epochs=UNFREEZE_EPOCH-1,
        callbacks=callbacks,
        verbose=1
    )

    # — Fine-tuning completo —
    opt2 = AdamW(
        learning_rate=FT_LR,
        weight_decay=WEIGHT_DECAY,
        beta_1=BETA_1,
        beta_2=BETA_2,
        epsilon=EPSILON
    )
    opt2 = LossScaleOptimizer(opt2)

    transformer_model.trainable = True
    model.compile(
        optimizer=opt2,
        loss="binary_crossentropy",
        metrics=[AUC(name="AUC", multi_label=True)]
    )
    model.fit(
        train_ds,
        validation_data=val_ds,
        initial_epoch=UNFREEZE_EPOCH-1,
        epochs=EPOCHS,
        callbacks=callbacks,
        verbose=1
    )

    # Evaluación de este fold
    m = model.evaluate(val_ds, return_dict=True)
    print(f"Fold {fold} — AUC:", m["AUC"])
    fold_aucs.append(m["AUC"])

# ── 4) Resultados finales de CV ──
print("\nAUC por fold:", fold_aucs)
print("Media ± std:", np.mean(fold_aucs), "±", np.std(fold_aucs))


>>> Fold 1


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFElectraModel: ['discriminator_predictions.dense.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.bias', 'electra.embeddings.position_ids', 'discriminator_predictions.dense_prediction.weight']
- This IS expected if you are initializing TFElectraModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFElectraModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFElectraModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFElectraModel for predictions without further train

NameError: name 'LossScaleOptimizer' is not defined